In [5]:
!pip install gensim


  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/e8/d9/104988573fd2c1acdc64e66883b35fb8ae559310d2d9f77db78bf7de9add/gensim-4.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 34.9 MB/s eta 0:00:00:00:0100:01


In [ ]:
#https://github.com/gweissman/clinical_embeddings

In [3]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText

import collections
import gc 

import warnings
# import glove
# from glove import Corpus
warnings.filterwarnings('ignore')

In [4]:
new_notes = pd.read_pickle("data/ner_df.p") # med7
w2vec = Word2Vec.load("embeddings/word2vec.model")
fasttext = FastText.load("embeddings/FT_100/ft_oa_all_100d.bin")
#fasttext = FastText.load("embeddings/fasttext.model")

In [5]:
null_index_list = []
for i in new_notes.itertuples():
    
    if len(i.ner) == 0:
        null_index_list.append(i.Index)
new_notes.drop(null_index_list, inplace=True)

In [4]:
med7_ner_data = {}

for ii in new_notes.itertuples():
    
    p_id = ii.SUBJECT_ID
    ind = ii.Index
    
    try:
        new_ner = new_notes.loc[ind].ner
    except:
        new_ner = []
            
    unique = set()
    new_temp = []
    
    for j in new_ner:
        for k in j:
            
            unique.add(k[0])
            new_temp.append(k)

    if p_id in med7_ner_data:
        for i in new_temp:
            med7_ner_data[p_id].append(i)
    else:
        med7_ner_data[p_id] = new_temp

In [5]:
len(med7_ner_data.keys())

22446

In [6]:
pd.to_pickle(med7_ner_data, "data/new_ner_word_dict.pkl")

In [11]:
def mean(a):
    return sum(a) / len(a)

In [36]:
data_types = [med7_ner_data]
data_names = ["new_ner"]

for data, names in zip(data_types, data_names):
    new_word2vec = {}
    print("w2vec starting..")
    for k,v in data.items():

        patient_temp = []
        for i in v:
            try:
                patient_temp.append(w2vec.wv[i[0]])
            except:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = w2vec.wv[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if num == 0: continue
                    avg = np.asarray(avg)
                    t = np.asarray(map(mean, zip(*avg)))
                    patient_temp.append(t)
        if len(patient_temp) == 0: continue
        new_word2vec[k] = patient_temp

    #############################################################################
    print("fasttext starting..")
        
    new_fasttextvec = {}

    for k,v in data.items():

        patient_temp = []

        for i in v:
            try:
                patient_temp.append(fasttext.wv[i[0]])
            except:
                pass
        if len(patient_temp) == 0: continue
        new_fasttextvec[k] = patient_temp

    #############################################################################    
        
    print("combined starting..")
    new_concatvec = {}

    for k,v in data.items():
        patient_temp = []
    #     if k != 6: continue
        for i in v:
            w2vec_temp = []
            try:
                w2vec_temp = w2vec.wv[i[0]]
            except:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = w2vec.wv[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if num == 0: 
                        w2vec_temp = [0] * 100
                    else:
                        avg = np.asarray(avg)
                        #w2vec_temp = np.asarray(map(mean, zip(*avg)))
                        w2vec_temp = np.asarray(list(map(mean, zip(*avg))))
                else:
                    w2vec_temp = [0] * 100

            #fasttemp = fasttext.wv[i[0]]
            try:
                fasttemp = fasttext.wv[i[0]]
            except IndexError:
                fasttemp = np.zeros(100)


            appended = np.append(fasttemp, w2vec_temp, 0)
            #appended = np.concatenate((fasttemp, w2vec_temp))

            patient_temp.append(appended)
        if len(patient_temp) == 0: continue
        new_concatvec[k] = patient_temp

    print(len(new_word2vec), len(new_fasttextvec), len(new_concatvec))
    pd.to_pickle(new_word2vec, "data/"+names+"_word2vec_dict.pkl")
    pd.to_pickle(new_fasttextvec, "data/"+names+"_fasttext_dict.pkl")
    pd.to_pickle(new_concatvec, "data/"+names+"_combined_dict.pkl")

w2vec starting..
fasttext starting..
combined starting..
22203 22446 22446


In [38]:
new_fasttextvec.keys()

dict_keys([27866, 29075, 29705, 29983, 31916, 29080, 27370, 29307, 28792, 28801, 29552, 27171, 31024, 27366, 28152, 29959, 27182, 30661, 29061, 29202, 29702, 29056, 29200, 28050, 26780, 31022, 29070, 31820, 29546, 31134, 27196, 30668, 27888, 30154, 28080, 29464, 31902, 28498, 31136, 30670, 27424, 30593, 29620, 28915, 26763, 31688, 30301, 29929, 29539, 29536, 28190, 26896, 30177, 28517, 29927, 29336, 28484, 30663, 26881, 31829, 29518, 30192, 29590, 29330, 31144, 28103, 27094, 29325, 28207, 28936, 30915, 30601, 27096, 30190, 31551, 29574, 28532, 29344, 28126, 28230, 31556, 31807, 30207, 31685, 30113, 29317, 30184, 30227, 29649, 11085, 30599, 29328, 26767, 29541, 28933, 30181, 29647, 28527, 28931, 29639, 28089, 27597, 31583, 27599, 27793, 29450, 31513, 31208, 31516, 26759, 29447, 27600, 30993, 30552, 29625, 31580, 29375, 30490, 31205, 27379, 31275, 31260, 31560, 29817, 27605, 30273, 27800, 27602, 29041, 29442, 30211, 30253, 29444, 29755, 27586, 31986, 30256, 27603, 27588, 29566, 27400, 30

In [40]:
diff = set(new_fasttextvec.keys()).difference(set(new_word2vec))
for i in diff:
    del new_fasttextvec[i]
    del new_concatvec[i]
print (len(new_word2vec), len(new_fasttextvec), len(new_concatvec))


pd.to_pickle(new_word2vec, "data/"+"new_ner"+"_word2vec_limited_dict.pkl")
pd.to_pickle(new_fasttextvec, "data/"+"new_ner"+"_fasttext_limited_dict.pkl")
pd.to_pickle(new_concatvec, "data/"+"new_ner"+"_combined_limited_dict.pkl")

22203 22203 22203
